<a href="https://colab.research.google.com/github/sindhura-nk/Artificial-Intelligence-1446/blob/main/Next_sentence_Word_Generation_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Buisness Scenario: Build a model that can predict the next words when some words are provided to it.

In [25]:
pip install nltk

In [26]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [27]:
# Import all the necessary libraries
from warnings import filterwarnings
filterwarnings(action='ignore')

# Preprocess the text
from tensorflow.keras.preprocessing.text import Tokenizer # creates work tokens, number sequences
from keras.preprocessing import sequence # padding
# Model
from keras.models import Sequential
from keras.layers import Input,Dense,LSTM,Embedding

## LOad the dataset

In [28]:
with open("/content/Harry Potter and the Sorcerer's Sto.txt",'r') as file:
  data = file.read()
print(data[:50])

Harry Potter and the Sorcerer's Stone 

CHAPTER ON


In [29]:
data[:50]

"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ON"

We are using Keras Tokenizer here. Few points are discussed below related to fit_on_texts method and texts_to_sequences method:
tokenizer.fit_on_texts(texts) : Builds the word index (vocabulary) from your dataset.

It looks at the list of texts you give it. It assigns a unique integer index to each unique word.
The most frequent word gets index 1, the next most frequent 2, and so on. Index 0 is reserved for padding.

Example: texts = ["The wizard cast a spell", "The spell was powerful"]
tokenizer.fit_on_texts(texts)
print(tokenizer.word_index)
Output: {'the': 1, 'spell': 2, 'wizard': 3, 'cast': 4, 'a': 5, 'was': 6, 'powerful': 7}
tokenizer.texts_to_sequences(texts): Converts your texts into sequences of integers based on the word index created by fit_on_texts.

Example: sequences = tokenizer.texts_to_sequences(["The wizard cast a spell"])
print(sequences)
Output: [[1, 3, 4, 5, 2]]

In [30]:
# Intializing the tokenizer
tokenizer = Tokenizer()

# apply the tokenizer on respective words
texts = ["The wizard cast a spell", "The spell was powerful"]
tokenizer.fit_on_texts(texts) # this is going to develop tokens and their frequency count
# the : 2
# wizard: 1
# cast: 1
# a : 1
# spell:2
print(tokenizer.word_index)
# word indexing: the:1, spell:2, wizard:3, cast:4, a:5.....


{'the': 1, 'spell': 2, 'wizard': 3, 'cast': 4, 'a': 5, 'was': 6, 'powerful': 7}


In [31]:
# if we want to give this text to the model
# [[1,3,4,5,2],[1,2,6,7]]
tokenizer.texts_to_sequences(texts)

[[1, 3, 4, 5, 2], [1, 2, 6, 7]]

In [32]:
# Intializing the tokenizer
tokenizer = Tokenizer()

# Fit on texts - pass the data
tokenizer.fit_on_texts([data])
word_ind = tokenizer.word_index
word_ind

{'the': 1,
 'and': 2,
 'to': 3,
 'a': 4,
 'he': 5,
 'of': 6,
 'harry': 7,
 'was': 8,
 'it': 9,
 'in': 10,
 'his': 11,
 'you': 12,
 'said': 13,
 'had': 14,
 'i': 15,
 'on': 16,
 'at': 17,
 'that': 18,
 'they': 19,
 'as': 20,
 'him': 21,
 'but': 22,
 'with': 23,
 'ron': 24,
 'all': 25,
 'out': 26,
 'for': 27,
 'up': 28,
 'be': 29,
 'what': 30,
 'hagrid': 31,
 'them': 32,
 'were': 33,
 'have': 34,
 'there': 35,
 'back': 36,
 'hermione': 37,
 'one': 38,
 'this': 39,
 'if': 40,
 'from': 41,
 'so': 42,
 'not': 43,
 'she': 44,
 'about': 45,
 'into': 46,
 'me': 47,
 'their': 48,
 'know': 49,
 'been': 50,
 'off': 51,
 'got': 52,
 'no': 53,
 'could': 54,
 "didn't": 55,
 'like': 56,
 'get': 57,
 'down': 58,
 'professor': 59,
 'just': 60,
 'her': 61,
 'see': 62,
 'who': 63,
 'when': 64,
 'is': 65,
 'very': 66,
 'over': 67,
 'looked': 68,
 'do': 69,
 'now': 70,
 'then': 71,
 'are': 72,
 'we': 73,
 'by': 74,
 "it's": 75,
 'your': 76,
 "don't": 77,
 'snape': 78,
 'around': 79,
 'dumbledore': 80,
 "he

In [33]:
reverse_word_index = {value:key for key,value in word_ind.items()}
reverse_word_index

{1: 'the',
 2: 'and',
 3: 'to',
 4: 'a',
 5: 'he',
 6: 'of',
 7: 'harry',
 8: 'was',
 9: 'it',
 10: 'in',
 11: 'his',
 12: 'you',
 13: 'said',
 14: 'had',
 15: 'i',
 16: 'on',
 17: 'at',
 18: 'that',
 19: 'they',
 20: 'as',
 21: 'him',
 22: 'but',
 23: 'with',
 24: 'ron',
 25: 'all',
 26: 'out',
 27: 'for',
 28: 'up',
 29: 'be',
 30: 'what',
 31: 'hagrid',
 32: 'them',
 33: 'were',
 34: 'have',
 35: 'there',
 36: 'back',
 37: 'hermione',
 38: 'one',
 39: 'this',
 40: 'if',
 41: 'from',
 42: 'so',
 43: 'not',
 44: 'she',
 45: 'about',
 46: 'into',
 47: 'me',
 48: 'their',
 49: 'know',
 50: 'been',
 51: 'off',
 52: 'got',
 53: 'no',
 54: 'could',
 55: "didn't",
 56: 'like',
 57: 'get',
 58: 'down',
 59: 'professor',
 60: 'just',
 61: 'her',
 62: 'see',
 63: 'who',
 64: 'when',
 65: 'is',
 66: 'very',
 67: 'over',
 68: 'looked',
 69: 'do',
 70: 'now',
 71: 'then',
 72: 'are',
 73: 'we',
 74: 'by',
 75: "it's",
 76: 'your',
 77: "don't",
 78: 'snape',
 79: 'around',
 80: 'dumbledore',
 81:

## Text on sequences

In [34]:
total_length = len(word_ind) + 1
# to reserve for padding

In [35]:
total_length

6032

In [36]:
input_sequences = []
for line in data.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  # [4,5,6,7]
  for i in range(1,len(token_list)): # for i in range(1,4)
    n_gram_sequence = token_list[:i+1] # token_list[0:2] = 0th index,1st index
    # token_list[0:3] = 0th index,1st index,2nd index
    # token_list[0:4] = 0th index, 1st index, 2nd index, 3rd index
    input_sequences.append(n_gram_sequence) # [[4,5],[4,5,6],[4,5,6,7]]
print(input_sequences[:15])

[[7, 121], [7, 121, 2], [7, 121, 2, 1], [7, 121, 2, 1, 634], [7, 121, 2, 1, 634, 158], [611, 38], [1, 144], [1, 144, 63], [1, 144, 63, 1049], [150, 2], [150, 2, 253], [150, 2, 253, 220], [150, 2, 253, 220, 6], [150, 2, 253, 220, 6, 612], [150, 2, 253, 220, 6, 612, 316]]


## padding to bring all the ngram tokens to same size. to bring all the setences to same length

In [37]:
max([90,89,77])

90

In [14]:
max_length = max([len(lines) for lines in input_sequences])

input_sequences = sequence.pad_sequences(input_sequences,maxlen=max_length)
input_sequences[:15]

array([[  0,   0,   0, ...,   0,   7, 121],
       [  0,   0,   0, ...,   7, 121,   2],
       [  0,   0,   0, ..., 121,   2,   1],
       ...,
       [  0,   0,   0, ..., 253, 220,   6],
       [  0,   0,   0, ..., 220,   6, 612],
       [  0,   0,   0, ...,   6, 612, 316]], dtype=int32)

In [15]:
print(max_length)

202


## Separate x and y

In [16]:
x = input_sequences[:,:-1]
y = input_sequences[:,-1]

In [17]:
x[:10]

array([[  0,   0,   0, ...,   0,   0,   7],
       [  0,   0,   0, ...,   0,   7, 121],
       [  0,   0,   0, ...,   7, 121,   2],
       ...,
       [  0,   0,   0, ...,   0,   1, 144],
       [  0,   0,   0, ...,   1, 144,  63],
       [  0,   0,   0, ...,   0,   0, 150]], dtype=int32)

In [18]:
y[:10]

array([ 121,    2,    1,  634,  158,   38,  144,   63, 1049,    2],
      dtype=int32)

In [19]:
y.shape

(74892,)

If you want to use categorical_crossentropy, then convert y into y cateogrical.
    y_cat = to_categorical(y)

## Model Buidling

In [20]:
model = Sequential()
# Provide the input as max length indicating total number of sentences
model.add(Input((max_length,)))
# Add the layers
model.add(Embedding(input_dim=total_length,output_dim=300,trainable=False))
model.add(LSTM(200,return_sequences=True,dropout=0.3)) # return sequences will provide the sequences to next LSTM layer
model.add(LSTM(150,dropout=0.2))
# Add one Hidden layer
model.add(Dense(100, activation='tanh'))
# Add output layer
model.add(Dense(total_length, activation='softmax'))


In [21]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='accuracy',patience=5)

In [23]:
nn = model.fit(x,y,validation_split=0.2,epochs=10,callbacks=[early_stop])

Epoch 1/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 61s 29ms/step - accuracy: 0.0452 - loss: 7.0019 - val_accuracy: 0.0441 - val_loss: 6.7468
Epoch 2/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 50s 27ms/step - accuracy: 0.0495 - loss: 6.5024 - val_accuracy: 0.0502 - val_loss: 6.6629
Epoch 3/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 49s 26ms/step - accuracy: 0.0596 - loss: 6.3304 - val_accuracy: 0.0628 - val_loss: 6.5066
Epoch 4/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.0705 - loss: 6.1156 - val_accuracy: 0.0694 - val_loss: 6.4172
Epoch 5/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.0812 - loss: 5.9305 - val_accuracy: 0.0793 - val_loss: 6.3275
Epoch 6/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.0865 - loss: 5.7609 - val_accuracy: 0.0807 - val_loss: 6.3176
Epoch 7/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.0919 - loss: 5.6114 - val_accuracy: 0.0899 - val_loss: 6.2823
Epoch 8/10
1873/1873 ━━━━━━━━━━━━━━━━━━━━ 48s 26ms/step - accuracy: 0.0963 -

In [39]:
import numpy as np
from nltk.tokenize import word_tokenize
def generate_text(user_text, next_words=10):
    for _ in range(next_words):
        # preparing our user text ready for the model
        token_list = [tokenizer.word_index.get(w, 0) for w in word_tokenize(user_text.lower())]
        token_list = sequence.pad_sequences([token_list],maxlen=max_length-1)
        # give the proceesed text to model for prediction of next 50words
        predicted_probs = model.predict(token_list.reshape(1, max_length-1), verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1) # it gives you the index of next word
        output_word = reverse_word_index.get(predicted[0], '') # convert the index into word using index_word
        user_text += ' ' + output_word
        # user_text = user_text + output_word
    return user_text

In [40]:
print(generate_text("The Wizard is"))

The Wizard is the door of the fat and the door of the
